In [24]:
!pip3 install --upgrade langchain langchain_experimental openai --quiet 

In [25]:
import os
import pandas as pd
from langchain.agents.agent_types import AgentType
from langchain_openai import OpenAI
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

In [26]:
# load csv
full_iseller_data = pd.read_excel('/Users/davidrinaldi/Documents/Projects/myk-dashboard/data/full_iseller_data.xlsx')
full_iseller_data.head()

,Unnamed: 0,base_price,buying_price,cancel_date,cancel_reason,cashier_id,channel,clean_product_name,closed_date,customer_id,day,device_location,fulfillment_status,hour,month,month_name,notes,on_account,order_id,order_type,outlet_code,outlet_name,payment_status,product_id,product_name,product_type,quantity,quarter,quarter_year,register_id,register_name,remaining_on_account,server_id,status,subtotal,tax_inclusive,total_amount,total_order_amount,total_shipping_amount,total_weight,week,week_of_month,year,cost_of_goods,profit
0,0,0,12348.0,NaT,NaN,f72af7ab-51ff-496f-b344-8cda0686df7e,pos,ayam! korean bites,2024-02-11 06:18:51,f4ee6240-51f7-4cc4-ba24-8a6acbf978ea,11,NaN,fulfilled,6,2,February,free baites,False,0da7e207-e9e6-418a-81d1-a9cb119c1f68,dine_in,Ayam! Libersa Park BKS,Ayam! Libersa Park BKS,paid,7aee5f23-c13b-4ea5-8c6b-a69739b504ca,ayam! korean bites,FOOD,1,1,Q1-2024,3a9262f8-0d6c-4de3-a0e0-033ce7ae70d2,Register #188 - samsung SM-T225,0,f72af7ab-51ff-496f-b344-8cda0686df7e,completed,15001,False,15001,15001,0,0,6,2,2024,1695.0,13306.0
1,1,0,13577.0,NaT,NaN,f72af7ab-51ff-496f-b344-8cda0686df7e,pos,ayam! korean bites,2024-02-04 07:11:14,b42f4d0d-00bc-432c-8f10-54167a53f0ac,4,NaN,fulfilled,6,2,February,free baites,False,4e213876-b632-4f95-be01-d7117e086a79,dine_in,Ayam! Libersa Park BKS,Ayam! Libersa Park BKS,paid,7aee5f23-c13b-4ea5-8c6b-a69739b504ca,ayam! korean bites,FOOD,1,1,Q1-2024,3a9262f8-0d6c-4de3-a0e0-033ce7ae70d2,Register #188 - samsung SM-T225,0,f72af7ab-51ff-496f-b344-8cda0686df7e,completed,25000,False,25000,25000,0,0,5,1,2024,20.0,24980.0
2,2,1,1493.0,NaT,NaN,13c394b6-2226-4fc3-b953-ca01c0a59d71,pos,free nasi putih,2024-02-09 12:50:47,6d5e2fcf-7732-40be-abda-04edfe52ebcf,9,NaN,fulfilled,12,2,February,NaN,False,cd4c3a55-84ed-4b16-9e5d-f5c9c354f94e,take_away,Bandung,Bandung,paid,cf8c3701-5943-4f45-83b1-3a7abc8f9827,free nasi putih,FOOD,1,1,Q1-2024,109a4272-036e-40f6-ad8d-61ad5ec5c353,Register #79 - samsung SM-X205,0,13c394b6-2226-4fc3-b953-ca01c0a59d71,completed,1,False,1,1,0,0,6,2,2024,1493.0,-1492.0
3,3,1,1493.0,NaT,NaN,7f7e33ff-b092-4f26-8c64-760362bf5bb5,pos,free nasi putih,2024-01-05 10:20:20,2a3d6390-4fd7-4bb0-b185-bdab64aeab75,5,NaN,fulfilled,10,1,January,NaN,False,712afc12-f2a2-4fb4-b527-8ffbe60ada21,take_away,Ayam! Dapur Kitchen BKS,Ayam! Dapur Kitchen BKS,paid,cf8c3701-5943-4f45-83b1-3a7abc8f9827,free nasi putih,FOOD,1,1,Q1-2024,2abf1638-c41a-4fef-88d8-35b2209c466c,Register #118 - samsung SM-T295,0,7f7e33ff-b092-4f26-8c64-760362bf5bb5,completed,1,False,1,1,0,0,1,1,2024,1493.0,-1492.0
4,4,1,1493.0,NaT,NaN,7f7e33ff-b092-4f26-8c64-760362bf5bb5,pos,free nasi putih,2024-01-12 14:04:16,30d5a638-2877-43a5-a402-3838efa52452,12,NaN,fulfilled,14,1,January,NaN,False,92da5a1e-667f-4213-ad67-5ab084dfb0e5,take_away,Ayam! Dapur Kitchen BKS,Ayam! Dapur Kitchen BKS,paid,cf8c3701-5943-4f45-83b1-3a7abc8f9827,free nasi putih,FOOD,1,1,Q1-2024,2abf1638-c41a-4fef-88d8-35b2209c466c,Register #118 - samsung SM-T295,0,7f7e33ff-b092-4f26-8c64-760362bf5bb5,completed,1,False,1,1,0,0,2,2,2024,1493.0,-1492.0


In [27]:
# initialise openai model
llm = OpenAI(
    temperature=0.8,
    top_p=0.7,
    model="gpt-3.5-turbo-instruct"
)

## location table

In [28]:
# initialise location tables
from scripts.utils import generate_location_df

location_df = generate_location_df(full_iseller_data)
location_df.head()

,outlet_name,year,month_name,month,quantity,quantity_diff,growth_rate
1,Ayam! Alun-Alun BKS,2024,March,3,172,NaN,NaN
0,Ayam! Alun-Alun BKS,2024,April,4,134,-38.0,-22.093023
2,Ayam! Alun-Alun BKS,2024,May,5,23,-111.0,-82.835821
5,Ayam! Dapur Kitchen BKS,2024,January,1,366,NaN,NaN
4,Ayam! Dapur Kitchen BKS,2024,February,2,331,-35.0,-9.562842


In [33]:
location_agent = create_pandas_dataframe_agent(
    llm=llm,
    df=location_df,
    verbose=True,
    allow_dangerous_code=True,
    handle_parsing_errors=True
)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_experimental/agents/agent_toolkits/pandas/base.py:282: UserWarning: Received additional kwargs {'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(


In [36]:
query = {"input": "Based on this data, generate three new and unique prompts that the user could ask in order to futher understand trends in the data."}
loc_response = location_agent.invoke(query)



> Entering new AgentExecutor chain...
Thought: We can use the column names to generate unique prompts that will help the user understand the data.
Action: python_repl_ast
Action Input: df.columnsIndex(['outlet_name', 'year', 'month_name', 'month', 'quantity',
       'quantity_diff', 'growth_rate'],
      dtype='object')Now we have a list of all the column names, we can use them to generate prompts.
Action: python_repl_ast
Action Input: df['outlet_name'].unique()['Ayam! Alun-Alun BKS' 'Ayam! Dapur Kitchen BKS' 'Ayam! Libersa Park BKS'
 'Ayam! RSIJ' 'Ayam! Tambun BKS' 'Bandung' 'Bekasi - Perjuangan' 'Cibubur'
 'Jakbar - Taman Palem' 'Jaksel - Pondok Indah' 'Jakut - Kelapa Gading'
 'Tangerang - Pisa Grande'] This prompt shows us the unique outlet names in the data, we can use this to ask about specific outlets.
Action: python_repl_ast
Action Input: df['quantity'].describe()count       53.000000
mean      1891.094340
std       2498.779797
min         20.000000
25%        172.000000
50%  

In [37]:
print(loc_response['output'])

Based on this data, some prompts that the user could ask to further understand trends are:
1. Can you show me the unique outlet names in the data?
2. How does the quantity differ across different months and outlets?
3. What is the overall trend in quantity for all outlets in this data?


In [38]:
prompt = 'Can you show me the unique outlet names in the data?'
print(location_agent.invoke(prompt)['output'])



> Entering new AgentExecutor chain...
Thought: I need to access the 'outlet_name' column and get the unique values.
Action: python_repl_ast
Action Input: df['outlet_name'].unique()['Ayam! Alun-Alun BKS' 'Ayam! Dapur Kitchen BKS' 'Ayam! Libersa Park BKS'
 'Ayam! RSIJ' 'Ayam! Tambun BKS' 'Bandung' 'Bekasi - Perjuangan' 'Cibubur'
 'Jakbar - Taman Palem' 'Jaksel - Pondok Indah' 'Jakut - Kelapa Gading'
 'Tangerang - Pisa Grande'] I now know the final answer
Final Answer: ['Ayam! Alun-Alun BKS' 'Ayam! Dapur Kitchen BKS' 'Ayam! Libersa Park BKS'
 'Ayam! RSIJ' 'Ayam! Tambun BKS' 'Bandung' 'Bekasi - Perjuangan' 'Cibubur'
 'Jakbar - Taman Palem' 'Jaksel - Pondok Indah' 'Jakut - Kelapa Gading'
 'Tangerang - Pisa Grande']

> Finished chain.
['Ayam! Alun-Alun BKS' 'Ayam! Dapur Kitchen BKS' 'Ayam! Libersa Park BKS'
 'Ayam! RSIJ' 'Ayam! Tambun BKS' 'Bandung' 'Bekasi - Perjuangan' 'Cibubur'
 'Jakbar - Taman Palem' 'Jaksel - Pondok Indah' 'Jakut - Kelapa Gading'
 'Tangerang - Pisa Grande']


In [41]:
prompt = 'How does the quantity differ across different months and outlets?'
plus = 'Create a markdown table to aid explain the result. Add a small summary explaining the result and recommendations'
print(location_agent.invoke(prompt+plus)['output'])



> Entering new AgentExecutor chain...
Thought: I need to group the data by outlet name and month, and then calculate the sum of the quantity for each group.
Action: python_repl_ast
Action Input: df.groupby(['outlet_name', 'month']).agg({'quantity': 'sum'})                               quantity
outlet_name             month          
Ayam! Alun-Alun BKS     3           172
                        4           134
                        5            23
Ayam! Dapur Kitchen BKS 1           366
                        2           331
                        3           524
                        4           662
                        5            46
Ayam! Libersa Park BKS  1           121
                        2           138
                        3            64
                        4            24
Ayam! RSIJ              1           297
                        2            20
                        3            95
Ayam! Tambun BKS        2            78
                      

In [42]:
prompt = 'What is the overall trend in quantity for all outlets in this data?'
plus = 'Create a markdown table to aid explain the result. Add a small summary explaining the result and recommendations'
print(location_agent.invoke(prompt+plus)['output'])



> Entering new AgentExecutor chain...
Thought: I need to calculate the sum of the quantity column for all outlets and compare it to the previous month's sum.
Action: python_repl_ast
Action Input: df['quantity'].sum()100228 I need to calculate the sum of the quantity column for all outlets and compare it to the previous month's sum.
Action: python_repl_ast
Action Input: df['quantity'].shift(1).sum()99817.0 I need to calculate the difference between the two sums and the growth rate.
Action: python_repl_ast
Action Input: df['quantity'].sum() - df['quantity'].shift(1).sum()411.0 I need to calculate the growth rate.
Action: python_repl_ast
Action Input: (df['quantity'].sum() - df['quantity'].shift(1).sum()) / df['quantity'].shift(1).sum() * 1000.411753508921326 I now know the final answer.
Final Answer: The overall trend in quantity for all outlets in this data is a slight increase of 0.41% from the previous month. This can be seen in the positive growth rate. It is recommended to continu

## sales table

In [11]:
# initialise sales tables
from scripts.utils import generate_sales_table

sales_df = generate_sales_table(full_iseller_data)
sales_df.head()

,clean_product_name,product_type,total_revenue,current_quarter_revenue,last_quarter_revenue,current_month_revenue,last_month_revenue,quarterly_growth,monthly_growth,quarter_Growth(%),Monthly_Growth(%)
0,(es batu) crystalline cup,REGULAR,201033098,55569433.0,145463665.0,10521000.0,45048433.0,-61.798410,-76.645137,-0.45,-0.62
1,a&w root beer kaleng,REGULAR,21235100,5086300.0,16148800.0,89000.0,4997300.0,-68.503542,-98.219038,-0.52,-0.97
2,absolut vodka,VODKA,51599900,18173000.0,33426900.0,2180000.0,15993000.0,-45.633606,-86.369036,-0.30,-0.76
3,alexis anggur hijau,ANGGUR,182694900,40367800.0,142327100.0,9386500.0,30981300.0,-71.637306,-69.702692,-0.56,-0.53
4,alexis anggur merah,ANGGUR,70802100,18252100.0,52550000.0,2918100.0,15334000.0,-65.267174,-80.969740,-0.48,-0.68


In [43]:
sales_agent = create_pandas_dataframe_agent(
    llm=llm,
    df=sales_df,
    verbose=True,
    allow_dangerous_code=True,
    handle_parsing_errors=True
)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_experimental/agents/agent_toolkits/pandas/base.py:282: UserWarning: Received additional kwargs {'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(


In [44]:
query = {"input": "Based on this data, generate three new and unique prompts that the user could ask in order to futher understand trends in the data."}
sales_response = location_agent.invoke(query)



> Entering new AgentExecutor chain...
Thought: We can use the prompts to get more specific insights about the data.
Action: [python_repl_ast]
Action Input: df.describe()[python_repl_ast] is not a valid tool, try one of [python_repl_ast].We can use the prompts to get more specific insights about the data.
Action: [python_repl_ast]
Action Input: df.groupby('outlet_name').sum()[python_repl_ast] is not a valid tool, try one of [python_repl_ast].We can use the prompts to get more specific insights about the data.
Action: [python_repl_ast]
Action Input: df.groupby('month_name').sum()[python_repl_ast] is not a valid tool, try one of [python_repl_ast]. I now know the final answer
Final Answer: Some possible prompts to ask about the data are:
1. What is the overall growth rate for each outlet?
2. How does the quantity of items sold differ between each month?
3. Which outlet had the highest quantity of items sold in a given month?

> Finished chain.


In [45]:
print(sales_response['output'])

Some possible prompts to ask about the data are:
1. What is the overall growth rate for each outlet?
2. How does the quantity of items sold differ between each month?
3. Which outlet had the highest quantity of items sold in a given month?


In [62]:
prompt = 'What is the overall growth rate for each outlet?'
plus = 'Add a natural language summary explaining the results and show ALL numerical values.'
print(location_agent.invoke(prompt+plus)['output'])



> Entering new AgentExecutor chain...
Thought: We need to calculate the overall growth rate for each outlet, which means we need to group the dataframe by outlet name and then calculate the growth rate for each group.
Action: python_repl_ast
Action Input: df.groupby('outlet_name')['growth_rate'].sum()outlet_name
Ayam! Alun-Alun BKS         0.000000
Ayam! Dapur Kitchen BKS     2.105821
Ayam! Libersa Park BKS    -20.223212
Ayam! RSIJ                  8.486027
Ayam! Tambun BKS           -0.095455
Bandung                    18.578317
Bekasi - Perjuangan        40.812687
Cibubur                    -8.362832
Jakbar - Taman Palem       -3.853160
Jaksel - Pondok Indah       4.953411
Jakut - Kelapa Gading      -2.584964
Tangerang - Pisa Grande    -3.077566
Name: growth_rate, dtype: float64 We can see that the overall growth rate for each outlet is shown in the 'growth_rate' column, grouped by the outlet name.
Action: python_repl_ast
Action Input: df.groupby('outlet_name')['growth_rate'].sum()

The overall growth rate for each outlet is shown in the table above. Ayam! Dapur Kitchen BKS has the highest average growth rate of 0.701940, while Ayam! Libersa Park BKS has the lowest average growth rate of -5.055803. This information can help us understand which outlets are experiencing the most growth and which ones may need improvement.


In [56]:
prompt = 'How does the quantity of items sold differ between each month?'
print(location_agent.invoke(prompt)['output'])



> Entering new AgentExecutor chain...
Thought: I need to group the dataframe by month and then calculate the difference between the quantity values.
Action: python_repl_ast
Action Input: df.groupby('month')['quantity'].diff()0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
5       352.0
6       528.0
7        23.0
8      -245.0
9      -193.0
10     -460.0
11     -638.0
12      176.0
13     -118.0
14       31.0
15       58.0
16      -46.0
17       22.0
18      -26.0
19      327.0
20      722.0
21      500.0
22      245.0
23    11048.0
24     9354.0
25     6859.0
26     7654.0
27     2677.0
28    -7926.0
29    -6434.0
30    -4803.0
31    -4900.0
32    -1627.0
33    -2126.0
34    -1443.0
35     -309.0
36    -1316.0
37     -461.0
38     2294.0
39     1492.0
40      323.0
41     1732.0
42      202.0
43    -1267.0
44    -1078.0
45     -323.0
46    -1019.0
47     -126.0
48     -777.0
49    -1178.0
50     -802.0
51     -904.0
52     -274.0
Name: quantity, dtype: float64 

In [50]:
prompt = 'Which outlet had the highest quantity of items sold in a given month?'
plus = 'Add a table or a list to explain the results. Add a small summary explaining the result and recommendations'
print(location_agent.invoke(prompt+plus)['output'])



> Entering new AgentExecutor chain...
Thought: I need to group the data by outlet_name and month, then find the sum of quantity for each group. Then, I can sort the data by the sum of quantity to find the outlet with the highest quantity.
Action: python_repl_ast
Action Input: df.groupby(['outlet_name', 'month_name'])['quantity'].sum().sort_values(ascending=False)outlet_name              month_name
Bekasi - Perjuangan      January       11672
                         February      10154
                         April          7945
                         March          7408
Jaksel - Pondok Indah    January        3914
                         February       3769
Cibubur                  January        3746
                         February       3720
Jaksel - Pondok Indah    April          3461
Cibubur                  April          3045
Bekasi - Perjuangan      May            2697
Jakut - Kelapa Gading    February       2691
                         January        2647
Jaksel - Pon